# Open Notebook CLI Documentation

This notebook provides comprehensive documentation for the Open Notebook CLI tool, explaining how to interact with notebooks, sources, notes, transformations, podcasts, and other features from the command line.

## Introduction

The Open Notebook CLI is a command-line interface that allows you to interact with all aspects of the Open Notebook application. It's particularly useful for:

- Automating workflows
- Bulk operations
- Integration with other tools and scripts
- Remote management of your Open Notebook instance

This tool must be run from within the Docker container where Open Notebook is installed.

## Getting Started

### Installation

The CLI script is located at `/app/open_notebook_cli.py` in the Docker container. You can run commands directly by connecting to the container and invoking the script.

### Basic Usage

Connect to the container and run a command:

```bash
# Connect to the Docker container
docker exec -it assistant-notebook-1 bash

# Run a command inside the container
python /app/open_notebook_cli.py list-notebooks
```

Alternatively, run commands directly without entering the container:

```bash
docker exec -it assistant-notebook-1 python /app/open_notebook_cli.py list-notebooks
```

### JSON Output

All commands support JSON output with the `--json` flag, making integration with other tools easy:

```bash
docker exec -it assistant-notebook-1 python /app/open_notebook_cli.py list-notebooks --json
```

## Notebook Commands

Commands for managing notebooks (collections of sources and notes).

In [ ]:
# List all notebooks
python open_notebook_cli.py list-notebooks

# List all notebooks including archived ones
python open_notebook_cli.py list-notebooks --include-archived

# List notebooks ordered by creation date
python open_notebook_cli.py list-notebooks --order-by "created desc"

# Get details of a specific notebook
python open_notebook_cli.py get-notebook notebook:abc123

# Create a new notebook
python open_notebook_cli.py create-notebook "Research Project" "Notes about my research on quantum computing"

# Archive a notebook
python open_notebook_cli.py archive-notebook notebook:abc123

# Unarchive a notebook
python open_notebook_cli.py unarchive-notebook notebook:abc123

## Source Commands

Commands for managing sources (documents, URLs, text content) in your notebooks.

In [ ]:
# List all sources in a notebook
python open_notebook_cli.py list-sources notebook:abc123

# Get details of a specific source
python open_notebook_cli.py get-source source:abc123

# Get source details including the full text
python open_notebook_cli.py get-source source:abc123 --full-text

# Get source details including insights
python open_notebook_cli.py get-source source:abc123 --show-insights

# Add a text source to a notebook
python open_notebook_cli.py add-text-source notebook:abc123 "Source Title" "This is the content of my source"

# Add a text source with embedding and transformations
python open_notebook_cli.py add-text-source notebook:abc123 "Source Title" "This is the content" --embed --apply-transformations transformation:abc123 transformation:def456

# Add a URL source to a notebook
python open_notebook_cli.py add-url-source notebook:abc123 https://example.com

# Generate embeddings for a source (for vector search)
python open_notebook_cli.py embed-source source:abc123

## Note Commands

Commands for managing notes in your notebooks.

In [ ]:
# List all notes in a notebook
python open_notebook_cli.py list-notes notebook:abc123

# Get details of a specific note
python open_notebook_cli.py get-note note:abc123

# Create a new note and add it to a notebook
python open_notebook_cli.py create-note notebook:abc123 "Note Title" "This is the content of my note"

# Create an AI-generated note
python open_notebook_cli.py create-note notebook:abc123 "AI Note" "This was generated by AI" --type ai

# Convert a source insight to a note
python open_notebook_cli.py insight-to-note source_insight:abc123 notebook:def456

## Transformation Commands

Commands for managing and applying transformations (AI-powered insights from sources).

In [ ]:
# List all transformations
python open_notebook_cli.py list-transformations

# Get details of a specific transformation
python open_notebook_cli.py get-transformation transformation:abc123

# Create a new transformation
python open_notebook_cli.py create-transformation "Key Points" "Key Points" "Extracts key points from text" "Extract the 5 most important points from this text and format them as a bulleted list"

# Create a transformation that's applied by default to new sources
python open_notebook_cli.py create-transformation "Summary" "Summary" "Summarizes text" "Provide a concise summary of the text in 3-5 sentences." --apply-default

# Apply a transformation to a source
python open_notebook_cli.py apply-transformation source:abc123 transformation:def456

## Chat Session Commands

Commands for managing chat sessions in notebooks.

In [ ]:
# List all chat sessions for a notebook
python open_notebook_cli.py list-chat-sessions notebook:abc123

# Create a new chat session for a notebook
python open_notebook_cli.py create-chat-session notebook:abc123 "Research Discussion"

## Search Commands

Commands for searching across your notebooks, sources, and notes.

In [ ]:
# Perform a text search across sources and notes
python open_notebook_cli.py text-search "quantum computing"

# Limit number of results
python open_notebook_cli.py text-search "quantum computing" --results 5

# Search only in sources or notes
python open_notebook_cli.py text-search "quantum computing" --source --no-note

# Perform a vector (semantic) search
python open_notebook_cli.py vector-search "how do quantum computers work"

# Adjust minimum similarity score for vector search
python open_notebook_cli.py vector-search "how do quantum computers work" --min-score 0.3

## Podcast Commands

Commands for managing podcast templates and generating podcasts from your content.

In [ ]:
# List all podcast templates
python open_notebook_cli.py list-podcast-templates

# Get details of a specific podcast template
python open_notebook_cli.py get-podcast-template podcast_config:abc123

# List all podcast episodes
python open_notebook_cli.py list-podcast-episodes

# Generate a podcast from a notebook
python open_notebook_cli.py generate-podcast podcast_config:abc123 notebook:def456 "Quantum Computing Insights"

# Generate a longer podcast with additional instructions
python open_notebook_cli.py generate-podcast podcast_config:abc123 notebook:def456 "Deep Dive into Quantum Computing" --longform --instructions "Focus on practical applications and recent breakthroughs"

## Example Workflows

This section demonstrates how to combine multiple commands for common tasks.

### Creating a Research Notebook with Sources

This example shows how to create a new notebook, add sources, and apply transformations.

In [ ]:
# Create a new notebook
NOTEBOOK_ID=$(python open_notebook_cli.py create-notebook "Quantum Computing Research" "My research on quantum computing principles and applications" --json | jq -r '.id')

# Add URL sources
python open_notebook_cli.py add-url-source "$NOTEBOOK_ID" "https://en.wikipedia.org/wiki/Quantum_computing" --embed
python open_notebook_cli.py add-url-source "$NOTEBOOK_ID" "https://quantum-computing.ibm.com/docs/guide/" --embed

# Get transformations to apply
TRANSFORM_IDS=$(python open_notebook_cli.py list-transformations --json | jq -r '.[] | select(.name=="Summary" or .name=="Key Points") | .id')

# Apply transformations to all sources in the notebook
for SOURCE_ID in $(python open_notebook_cli.py list-sources "$NOTEBOOK_ID" --json | jq -r '.[].id'); do
  for TRANSFORM_ID in $TRANSFORM_IDS; do
    python open_notebook_cli.py apply-transformation "$SOURCE_ID" "$TRANSFORM_ID"
  done
done

# Create a chat session for the notebook
python open_notebook_cli.py create-chat-session "$NOTEBOOK_ID" "Initial Research Discussion"

### Generating a Podcast from Research

This example shows how to search for relevant content, create a focused notebook, and generate a podcast.

In [ ]:
# Create a new focused notebook
PODCAST_NOTEBOOK_ID=$(python open_notebook_cli.py create-notebook "Quantum Computing Podcast" "Content for my quantum computing podcast episode" --json | jq -r '.id')

# Search for relevant content across all notebooks
SEARCH_RESULTS=$(python open_notebook_cli.py vector-search "practical applications of quantum computing" --results 10 --json)

# Extract source IDs from search results
SOURCE_IDS=$(echo "$SEARCH_RESULTS" | jq -r '.[] | select(.type=="source") | .id')

# Extract note IDs from search results
NOTE_IDS=$(echo "$SEARCH_RESULTS" | jq -r '.[] | select(.type=="note") | .id')

# Get podcast template ID
TEMPLATE_ID=$(python open_notebook_cli.py list-podcast-templates --json | jq -r '.[0].id')

# Generate the podcast
python open_notebook_cli.py generate-podcast "$TEMPLATE_ID" "$PODCAST_NOTEBOOK_ID" "Quantum Computing: Applications and Future" --longform --instructions "Focus on practical applications and future developments in quantum computing. Make it accessible to a general audience."

## Advanced Usage

### Scripting and Automation

The CLI tool is designed to be easily integrated into scripts and automation workflows. You can:

1. Use the `--json` flag to get structured output
2. Pipe commands together
3. Use in shell scripts or cron jobs
4. Integrate with other tools via JSON output

### Error Handling

The tool provides clear error messages when operations fail. Common errors include:

- Item not found (invalid ID)
- Invalid input (missing required fields)
- Permission issues

Always check for error messages in script automation.

## Command Reference Summary

Here's a quick reference of all available commands:

### Notebook Commands
- `list-notebooks`: List all notebooks
- `get-notebook`: Get a specific notebook
- `create-notebook`: Create a new notebook
- `archive-notebook`: Archive a notebook
- `unarchive-notebook`: Unarchive a notebook

### Source Commands
- `list-sources`: List sources in a notebook
- `get-source`: Get details of a source
- `add-text-source`: Add a text source
  - Use `--transform` to apply standard transformations and enable embedding
- `add-url-source`: Add a URL source
  - Use `--transform` to apply standard transformations and enable embedding
- `embed-source`: Generate embeddings for a source

### Note Commands
- `list-notes`: List notes in a notebook
- `get-note`: Get details of a note
- `create-note`: Create a new note
- `insight-to-note`: Convert source insight to note

### Transformation Commands
- `list-transformations`: List all transformations
- `get-transformation`: Get details of a transformation
- `create-transformation`: Create a new transformation
- `apply-transformation`: Apply transformation to source
  - Use `--transform` to apply all standard transformations

### Chat Commands
- `list-chat-sessions`: List chat sessions for a notebook
- `create-chat-session`: Create a new chat session

### Search Commands
- `text-search`: Perform text search
- `vector-search`: Perform semantic vector search

### Podcast Commands
- `list-podcast-templates`: List podcast templates
- `get-podcast-template`: Get details of a podcast template
- `list-podcast-episodes`: List podcast episodes
- `generate-podcast`: Generate a podcast from a notebook

## Batch Transformations with the --transform Parameter

The CLI provides a convenient way to apply multiple standard transformations at once using the `--transform` parameter. This parameter is a shortcut that automatically applies a carefully selected set of AI transformations to your content and enables embedding for vector search.

Using the `--transform` parameter provides several benefits:

1. **Efficiency**: Apply all standard transformations with a single command
2. **Consistency**: Ensure all content is processed the same way
3. **Automation**: The `--transform` parameter automatically enables embedding
4. **Comprehensiveness**: Get multiple AI-powered insights for each source

This approach is particularly useful when adding new sources that you want to process with a comprehensive set of analyses.

In [ ]:
# Apply multiple specific transformations with the --transform parameter
python open_notebook_cli.py add-text-source notebook:abc123 "Research Paper" "This is the content of my research paper" --transform

# Apply the same transformations to a URL source
python open_notebook_cli.py add-url-source notebook:abc123 https://example.com/research-paper --transform

# Apply the same transformations to an existing source
python open_notebook_cli.py apply-transformation source:abc123 --transform

### Transformation ID Reference

The CLI has a built-in set of standard transformations that are applied when using the `--transform` parameter. These transformations are:

- `transformation:six06lwh4za956v8b53y`: Summary transformation
- `transformation:75oaeqdl73m3vk31sc66`: Key Points extraction
- `transformation:q2g9i1ijrr37a9ubopku`: Questions and Answers generation
- `transformation:5tvf3lsffw5onsh4eecr`: Critical Analysis
- `transformation:peilrk93qprsggk4pck4`: Potential Applications
- `transformation:9d74z8nhwml1c9zi04ng`: Further Research Suggestions

When using the `--transform` parameter, the system automatically:

1. Applies all standard transformations to the source
2. Enables embedding for vector search
3. Processes all transformations in parallel

This is much more efficient than applying transformations individually.

### Scripting with Batch Transformations

This example shows how to use batch transformations in a script to efficiently process a new source, check its transformation results, and then use that information in downstream processing:

In [ ]:
# Create a new notebook
NOTEBOOK_ID=$(python open_notebook_cli.py create-notebook "Advanced Research Topic" "Comprehensive research with multiple AI insights" --json | jq -r '.id')

# Add a source with the standard transformations and embeddings
SOURCE_ID=$(python open_notebook_cli.py add-url-source "$NOTEBOOK_ID" "https://example.com/advanced-research" --transform --json | jq -r '.id')

# Check the status of the transformations
python open_notebook_cli.py get-source "$SOURCE_ID" --show-insights

# Check a specific transformation result (Summary)
python open_notebook_cli.py get-source "$SOURCE_ID" --show-insights | grep -A 5 "Summary"